# Finance & Analytics club

Choose any group of stocks and stich to this group as a source for your portfolio for the entire duration of your backtesting

Build fixed individual position sized long only portfolioby picking m number of stocks based on monthly returns

Rebalance the portfolio every month by removing worse x stocks and replacing them with top x stocks from the universe of stocks.(its your decision to pick a stock twice in the same portfolio)

Backtest the strategy and compare the KPIs with that of simple buy and hold strategy of corresponding index.



In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import copy
import matplotlib.pyplot as plt

### Function to calculate the Cumulative Annual Growth Rate of a trading strategy


In [2]:
def CAGR(DF):
    df = DF.copy()
    df["cum_return"] = (1 + df["mon_ret"]).cumprod()
    n = len(df)/12
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

### Function to calculate annualized volatility of a trading strategy

In [3]:
def volatility(DF):
    df = DF.copy()
    vol = df["mon_ret"].std() * np.sqrt(12)
    return vol

### Function to calculate sharpe ratio ; rf is the risk free rate

In [4]:
def sharpe(DF,rf):
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr

### Function to calculate max drawdown

In [5]:
def max_dd(DF):
    df = DF.copy()
    df["cum_return"] = (1 + df["mon_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

### Download historical data 

In [6]:
tickers = ["IDEA.NS","SBIN.NS","ITC.NS","ICICIBANK.NS","JINDALSTEL.NS","YESBANK.NS","RBLBANK.NS","AXISBANK.NS",
           "ONGC.NS","ASHOKLEY.NS","SUNPHARMA.NS","VEDL.NS","INDUSINDBK.NS","NCC.NS","BANKBARODA.NS","PNB.NS","IDFCFIRSTB.NS",
           "GAIL.NS","RELIANCE.NS","CIPLA.NS"]

In [7]:
ohlc_mon = {} # directory with ohlc value for each stock            
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            ohlc_mon[tickers[i]] = pdr.get_data_yahoo(tickers[i],datetime.date.today()-datetime.timedelta(1900),datetime.date.today(),interval='m')
            ohlc_mon[tickers[i]].dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1
 
tickers = ohlc_mon.keys() # redefine tickers variable after removing any tickers with corrupted data

### Calculating monthly return for each stock and consolidating return info by stock in a separate dataframe

In [8]:
ohlc_dict = copy.deepcopy(ohlc_mon)
return_df = pd.DataFrame()
for ticker in tickers:
    print("calculating monthly return for ",ticker)
    ohlc_dict[ticker]["mon_ret"] = ohlc_dict[ticker]["Adj Close"].pct_change()
    return_df[ticker] = ohlc_dict[ticker]["mon_ret"]
    
return_df

calculating monthly return for  IDEA.NS
calculating monthly return for  SBIN.NS
calculating monthly return for  ITC.NS
calculating monthly return for  ICICIBANK.NS
calculating monthly return for  JINDALSTEL.NS
calculating monthly return for  YESBANK.NS
calculating monthly return for  RBLBANK.NS
calculating monthly return for  AXISBANK.NS
calculating monthly return for  ONGC.NS
calculating monthly return for  ASHOKLEY.NS
calculating monthly return for  SUNPHARMA.NS
calculating monthly return for  VEDL.NS
calculating monthly return for  INDUSINDBK.NS
calculating monthly return for  NCC.NS
calculating monthly return for  BANKBARODA.NS
calculating monthly return for  PNB.NS
calculating monthly return for  IDFCFIRSTB.NS
calculating monthly return for  GAIL.NS
calculating monthly return for  RELIANCE.NS
calculating monthly return for  CIPLA.NS


,IDEA.NS,SBIN.NS,ITC.NS,ICICIBANK.NS,JINDALSTEL.NS,YESBANK.NS,RBLBANK.NS,AXISBANK.NS,ONGC.NS,ASHOKLEY.NS,SUNPHARMA.NS,VEDL.NS,INDUSINDBK.NS,NCC.NS,BANKBARODA.NS,PNB.NS,IDFCFIRSTB.NS,GAIL.NS,RELIANCE.NS,CIPLA.NS
Date,,,,,,,,,,,,,,,,,,,,
2015-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.098345,NaN,NaN,NaN,NaN
2015-03-31,-0.048907,0.010110,-0.010740,0.049603,0.044962,0.029053,NaN,0.013567,0.011638,-0.050478,-0.083065,0.107388,-0.070203,-0.188755,0.033639,0.106302,NaN,-0.070918,0.044007,-0.107306
2015-04-30,-0.010292,0.031140,0.014737,-0.041974,-0.269759,0.051045,NaN,0.030997,0.085031,0.026580,0.029025,-0.063855,0.060876,-0.094060,-0.037870,-0.038185,NaN,0.079243,0.016989,0.022720
2015-05-31,0.017035,-0.043441,-0.036535,-0.029156,-0.283744,0.008287,NaN,-0.045183,-0.061391,0.015395,-0.094711,-0.113515,-0.001430,-0.044930,-0.113469,-0.096648,NaN,0.008613,0.167173,-0.053886
2015-06-30,-0.015893,0.027783,0.065783,0.000422,-0.078180,-0.017657,NaN,0.026031,-0.119186,0.172146,-0.059684,-0.262130,0.120316,0.048951,0.259542,0.115406,NaN,-0.094975,0.001750,0.152178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,-0.138211,-0.045843,-0.010728,-0.024587,0.048897,-0.164004,-0.076990,-0.032887,-0.154115,0.005521,0.004046,-0.095441,-0.166325,-0.022321,-0.090285,-0.060606,-0.098560,-0.004957,-0.067633,-0.065454
2020-01-31,-0.169811,-0.048516,-0.117797,-0.054028,0.099488,-0.118471,-0.086082,-0.043878,-0.156035,-0.147041,-0.141377,-0.173314,-0.122969,-0.349772,-0.176915,-0.254756,-0.095823,-0.133250,-0.058796,-0.100246
2020-02-29,-0.295455,-0.350330,-0.172331,-0.348919,-0.574974,-0.351156,-0.534204,-0.456475,-0.257205,-0.384120,-0.048422,-0.432456,-0.681808,-0.473315,-0.298165,-0.281909,-0.426630,-0.226360,-0.161743,0.051604


### Function to calculate portfolio return iteratively

In [9]:
def pflio(DF,m,x):
    df = DF.copy()
    portfolio = []
    monthly_ret = [0]
    for i in range(1,len(df)):
        if len(portfolio) > 0:
            monthly_ret.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio = portfolio + new_picks
        print(portfolio)
    monthly_ret_df = pd.DataFrame(np.array(monthly_ret),columns=["mon_ret"])
    return monthly_ret_df

### Calculating overall strategy's KPIs

In [10]:
CAGR(pflio(return_df,6,3))

['VEDL.NS', 'PNB.NS', 'ICICIBANK.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'BANKBARODA.NS']
['PNB.NS', 'RELIANCE.NS', 'BANKBARODA.NS', 'ONGC.NS', 'GAIL.NS', 'INDUSINDBK.NS']
['RELIANCE.NS', 'GAIL.NS', 'INDUSINDBK.NS', 'RELIANCE.NS', 'IDEA.NS', 'ASHOKLEY.NS']
['INDUSINDBK.NS', 'ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'CIPLA.NS', 'INDUSINDBK.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'YESBANK.NS', 'RELIANCE.NS', 'ITC.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS']
['INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS', 'JINDALSTEL.NS', 'VEDL.NS', 'RELIANCE.NS']
['GAIL.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'GAIL.NS', 'PNB.NS']
['JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'JINDALSTEL.NS']
['RELIANCE.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'SUNPHARMA.NS', 'YESBANK.NS', 'RELIANCE.NS']
['SUNPHARMA.NS', 'SUNPHARMA.NS', 'BANKBARODA.NS', 'IDEA.NS', 'VEDL.NS', 'ASHOKLEY.NS']
['BANKBARODA.NS', 'VEDL.NS', 'ASHOKLEY.NS', 'V

-0.14628732869970862

In [11]:
sharpe(pflio(return_df,6,3),0.025)

['VEDL.NS', 'PNB.NS', 'ICICIBANK.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'BANKBARODA.NS']
['PNB.NS', 'RELIANCE.NS', 'BANKBARODA.NS', 'ONGC.NS', 'GAIL.NS', 'INDUSINDBK.NS']
['RELIANCE.NS', 'GAIL.NS', 'INDUSINDBK.NS', 'RELIANCE.NS', 'IDEA.NS', 'ASHOKLEY.NS']
['INDUSINDBK.NS', 'ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'CIPLA.NS', 'INDUSINDBK.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'YESBANK.NS', 'RELIANCE.NS', 'ITC.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS']
['INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS', 'JINDALSTEL.NS', 'VEDL.NS', 'RELIANCE.NS']
['GAIL.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'GAIL.NS', 'PNB.NS']
['JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'JINDALSTEL.NS']
['RELIANCE.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'SUNPHARMA.NS', 'YESBANK.NS', 'RELIANCE.NS']
['SUNPHARMA.NS', 'SUNPHARMA.NS', 'BANKBARODA.NS', 'IDEA.NS', 'VEDL.NS', 'ASHOKLEY.NS']
['BANKBARODA.NS', 'VEDL.NS', 'ASHOKLEY.NS', 'V

-0.5437879338701274

In [12]:
max_dd(pflio(return_df,6,3)) 

['VEDL.NS', 'PNB.NS', 'ICICIBANK.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'BANKBARODA.NS']
['PNB.NS', 'RELIANCE.NS', 'BANKBARODA.NS', 'ONGC.NS', 'GAIL.NS', 'INDUSINDBK.NS']
['RELIANCE.NS', 'GAIL.NS', 'INDUSINDBK.NS', 'RELIANCE.NS', 'IDEA.NS', 'ASHOKLEY.NS']
['INDUSINDBK.NS', 'ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'CIPLA.NS', 'INDUSINDBK.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'YESBANK.NS', 'RELIANCE.NS', 'ITC.NS']
['ASHOKLEY.NS', 'BANKBARODA.NS', 'ASHOKLEY.NS', 'INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS']
['INDUSINDBK.NS', 'GAIL.NS', 'NCC.NS', 'JINDALSTEL.NS', 'VEDL.NS', 'RELIANCE.NS']
['GAIL.NS', 'JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'GAIL.NS', 'PNB.NS']
['JINDALSTEL.NS', 'RELIANCE.NS', 'JINDALSTEL.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'JINDALSTEL.NS']
['RELIANCE.NS', 'SUNPHARMA.NS', 'ONGC.NS', 'SUNPHARMA.NS', 'YESBANK.NS', 'RELIANCE.NS']
['SUNPHARMA.NS', 'SUNPHARMA.NS', 'BANKBARODA.NS', 'IDEA.NS', 'VEDL.NS', 'ASHOKLEY.NS']
['BANKBARODA.NS', 'VEDL.NS', 'ASHOKLEY.NS', 'V

0.6528186663937235

### Above was the portfolio of random stocks, and it didn't worked well, lets take another portfolio with only healthcare sector stocks!

In [13]:
tickers = ["LALPATHLAB.BO","DIVISLAB.BO","BIOCON.BO","LUPIN.BO","CIPLA.BO","SUNPHARMA.BO","AUROPHARMA.BO","JUBILANT.BO",
           "WOCKPHARMA.BO","DRREDDY.BO","CADILAHC.BO","FORTIS.BO","SUVEN.BO","MOREPENLAB.BO","STAR.BO"]

In [14]:
ohlc_mon = {} # directory with ohlc value for each stock            
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            ohlc_mon[tickers[i]] = pdr.get_data_yahoo(tickers[i],datetime.date.today()-datetime.timedelta(1900),datetime.date.today(),interval='m')
            ohlc_mon[tickers[i]].dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1
 
tickers = ohlc_mon.keys() # redefine tickers variable after removing any tickers with corrupted data

In [15]:
ohlc_dict = copy.deepcopy(ohlc_mon)
return_df = pd.DataFrame()
for ticker in tickers:
    print("calculating monthly return for ",ticker)
    ohlc_dict[ticker]["mon_ret"] = ohlc_dict[ticker]["Adj Close"].pct_change()
    return_df[ticker] = ohlc_dict[ticker]["mon_ret"]
    
return_df

calculating monthly return for  LALPATHLAB.BO
calculating monthly return for  DIVISLAB.BO
calculating monthly return for  BIOCON.BO
calculating monthly return for  LUPIN.BO
calculating monthly return for  CIPLA.BO
calculating monthly return for  SUNPHARMA.BO
calculating monthly return for  AUROPHARMA.BO
calculating monthly return for  JUBILANT.BO
calculating monthly return for  WOCKPHARMA.BO
calculating monthly return for  DRREDDY.BO
calculating monthly return for  CADILAHC.BO
calculating monthly return for  FORTIS.BO
calculating monthly return for  SUVEN.BO
calculating monthly return for  MOREPENLAB.BO
calculating monthly return for  STAR.BO


,LALPATHLAB.BO,DIVISLAB.BO,BIOCON.BO,LUPIN.BO,CIPLA.BO,SUNPHARMA.BO,AUROPHARMA.BO,JUBILANT.BO,WOCKPHARMA.BO,DRREDDY.BO,CADILAHC.BO,FORTIS.BO,SUVEN.BO,MOREPENLAB.BO,STAR.BO
Date,,,,,,,,,,,,,,,
2015-11-30,NaN,0.007061,0.122170,0.026245,0.008540,0.122220,0.080669,-0.004610,-0.076756,-0.002170,-0.178549,0.087481,0.007778,0.398148,-0.060942
2015-12-31,-0.029544,-0.012118,-0.052987,-0.069404,-0.098230,0.064699,-0.048149,-0.149646,-0.168854,0.000145,-0.070470,-0.012205,-0.167917,-0.153642,-0.094566
2016-01-31,0.060694,-0.167054,-0.044131,0.024499,-0.120710,-0.020561,-0.213741,-0.118085,-0.401996,-0.023312,0.033147,-0.062061,-0.310865,-0.201878,-0.136174
2016-02-29,0.115955,0.036240,0.028042,-0.155181,-0.005437,-0.041635,0.138346,0.353104,0.278694,0.005278,0.006830,0.050599,0.265929,0.023529,0.079629
2016-03-31,0.075421,0.076422,0.296667,0.042966,0.048418,-0.009946,0.021409,-0.039150,0.021836,0.015440,0.041562,-0.005130,0.199455,-0.021073,0.075116
2016-04-30,-0.052170,0.050881,0.223803,-0.044985,-0.120298,-0.059904,0.032328,-0.096238,-0.076629,0.029118,0.027982,-0.072759,0.011912,-0.082192,-0.049017
2016-05-31,-0.099755,0.007752,0.033634,0.042695,0.061494,0.001180,-0.053657,-0.128889,0.016010,0.062524,-0.024691,-0.023170,-0.049389,0.031983,0.014061
2016-06-30,0.147706,0.079401,0.118736,0.132306,0.053246,0.086629,0.066744,0.064137,0.081091,-0.131890,0.116517,0.083808,-0.014322,-0.014463,0.030087
2016-07-31,0.105902,0.104651,0.106619,-0.143916,0.084162,-0.066164,-0.002083,0.602715,-0.189430,0.063650,0.039886,0.038809,-0.052219,-0.161426,-0.112399


In [16]:
CAGR(pflio(return_df,6,3))

['SUNPHARMA.BO', 'DRREDDY.BO', 'DIVISLAB.BO', 'FORTIS.BO', 'LALPATHLAB.BO', 'AUROPHARMA.BO']
['SUNPHARMA.BO', 'DRREDDY.BO', 'LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'LUPIN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'JUBILANT.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'SUVEN.BO', 'BIOCON.BO', 'SUVEN.BO', 'DIVISLAB.BO']
['BIOCON.BO', 'DIVISLAB.BO', 'BIOCON.BO', 'DIVISLAB.BO', 'AUROPHARMA.BO', 'DRREDDY.BO']
['BIOCON.BO', 'BIOCON.BO', 'DRREDDY.BO', 'DRREDDY.BO', 'CIPLA.BO', 'LUPIN.BO']
['BIOCON.BO', 'BIOCON.BO', 'LUPIN.BO', 'LALPATHLAB.BO', 'LUPIN.BO', 'BIOCON.BO']
['BIOCON.BO', 'BIOCON.BO', 'BIOCON.BO', 'JUBILANT.BO', 'BIOCON.BO', 'LALPATHLAB.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'AUROPHARMA.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'JUBILANT.BO', 'CADILAHC.BO', 'LALPATHLAB.BO']
['MOREPENLAB.BO', 'STAR.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'LUPIN.BO', 'LALPATHLAB.BO']
['STAR.BO', 'LUPIN.BO'

0.19633489010842253

In [17]:
sharpe(pflio(return_df,6,3),0.025)

['SUNPHARMA.BO', 'DRREDDY.BO', 'DIVISLAB.BO', 'FORTIS.BO', 'LALPATHLAB.BO', 'AUROPHARMA.BO']
['SUNPHARMA.BO', 'DRREDDY.BO', 'LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'LUPIN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'JUBILANT.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'SUVEN.BO', 'BIOCON.BO', 'SUVEN.BO', 'DIVISLAB.BO']
['BIOCON.BO', 'DIVISLAB.BO', 'BIOCON.BO', 'DIVISLAB.BO', 'AUROPHARMA.BO', 'DRREDDY.BO']
['BIOCON.BO', 'BIOCON.BO', 'DRREDDY.BO', 'DRREDDY.BO', 'CIPLA.BO', 'LUPIN.BO']
['BIOCON.BO', 'BIOCON.BO', 'LUPIN.BO', 'LALPATHLAB.BO', 'LUPIN.BO', 'BIOCON.BO']
['BIOCON.BO', 'BIOCON.BO', 'BIOCON.BO', 'JUBILANT.BO', 'BIOCON.BO', 'LALPATHLAB.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'AUROPHARMA.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'JUBILANT.BO', 'CADILAHC.BO', 'LALPATHLAB.BO']
['MOREPENLAB.BO', 'STAR.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'LUPIN.BO', 'LALPATHLAB.BO']
['STAR.BO', 'LUPIN.BO'

0.13572511646127977

In [18]:
max_dd(pflio(return_df,6,3)) 

['SUNPHARMA.BO', 'DRREDDY.BO', 'DIVISLAB.BO', 'FORTIS.BO', 'LALPATHLAB.BO', 'AUROPHARMA.BO']
['SUNPHARMA.BO', 'DRREDDY.BO', 'LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'LUPIN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'CADILAHC.BO', 'JUBILANT.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['LALPATHLAB.BO', 'LALPATHLAB.BO', 'SUVEN.BO', 'BIOCON.BO', 'SUVEN.BO', 'DIVISLAB.BO']
['BIOCON.BO', 'DIVISLAB.BO', 'BIOCON.BO', 'DIVISLAB.BO', 'AUROPHARMA.BO', 'DRREDDY.BO']
['BIOCON.BO', 'BIOCON.BO', 'DRREDDY.BO', 'DRREDDY.BO', 'CIPLA.BO', 'LUPIN.BO']
['BIOCON.BO', 'BIOCON.BO', 'LUPIN.BO', 'LALPATHLAB.BO', 'LUPIN.BO', 'BIOCON.BO']
['BIOCON.BO', 'BIOCON.BO', 'BIOCON.BO', 'JUBILANT.BO', 'BIOCON.BO', 'LALPATHLAB.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'AUROPHARMA.BO', 'WOCKPHARMA.BO', 'SUVEN.BO']
['JUBILANT.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'JUBILANT.BO', 'CADILAHC.BO', 'LALPATHLAB.BO']
['MOREPENLAB.BO', 'STAR.BO', 'MOREPENLAB.BO', 'JUBILANT.BO', 'LUPIN.BO', 'LALPATHLAB.BO']
['STAR.BO', 'LUPIN.BO'

0.5854446834673344

## We successfully got better results now!